# Ensemble Learning

## Initial Imports

In [193]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [194]:
import warnings
warnings.filterwarnings('ignore')

In [195]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [196]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [197]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [198]:
# Create our features
X = df.copy()
X.drop(["home_ownership","verification_status","loan_status","pymnt_plan","hardship_flag","debt_settlement_flag","issue_d","initial_list_status"], axis=1, inplace=True)
X =  pd.get_dummies(X)
X.head()

# Create our target
y = df["loan_status"].values.reshape(-1,1)
y[:5]

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [199]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,1.000000,1.000000,1.000000,1.000000


In [200]:
# Check the balance of our target values
y

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ...,
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [201]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state = 1,
                                                   stratify = y)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51612 entries, 39692 to 44756
Data columns (total 80 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   loan_amnt                    51612 non-null  float64
 1   int_rate                     51612 non-null  float64
 2   installment                  51612 non-null  float64
 3   annual_inc                   51612 non-null  float64
 4   dti                          51612 non-null  float64
 5   delinq_2yrs                  51612 non-null  float64
 6   inq_last_6mths               51612 non-null  float64
 7   open_acc                     51612 non-null  float64
 8   pub_rec                      51612 non-null  float64
 9   revol_bal                    51612 non-null  float64
 10  total_acc                    51612 non-null  float64
 11  out_prncp                    51612 non-null  float64
 12  out_prncp_inv                51612 non-null  float64
 13  total_pymnt 

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [202]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [203]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [204]:
# Scale the training and testing data
#X_scaler = 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [205]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [206]:
y_pred_rf = brf.predict(X_test)

In [207]:
# Calculated the balanced accuracy score
#from sklearn.metrics import balanced_accuracy_score
#balanced_accuracy_score(y_test, y_pred_rf)

y_pred = brf.predict(X_test_scaled)
brf_score = balanced_accuracy_score(y_test, y_pred)
print(brf_score)


0.5


In [208]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   53,    34],
       [ 2817, 14301]], dtype=int64)

In [209]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
brf_report=classification_report_imbalanced(y_test, y_pred)
print(brf_report)


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.61      0.84      0.04      0.71      0.50        87
   low_risk       1.00      0.84      0.61      0.91      0.71      0.52     17118

avg / total       0.99      0.83      0.61      0.90      0.71      0.52     17205



In [210]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
features_rank


[(0.08371932217726707, 'total_rec_prncp'),
 (0.06884763824889792, 'last_pymnt_amnt'),
 (0.056229790627804144, 'total_pymnt'),
 (0.054376893954612794, 'total_rec_int'),
 (0.05324054210916487, 'total_pymnt_inv'),
 (0.027251570941765165, 'int_rate'),
 (0.01956065388308564, 'bc_util'),
 (0.01921280075867695, 'max_bal_bc'),
 (0.019098015957605033, 'il_util'),
 (0.018860895306201334, 'installment'),
 (0.0180120992415877, 'out_prncp'),
 (0.01730306263569184, 'annual_inc'),
 (0.01712681149527115, 'revol_bal'),
 (0.01705813881984467, 'mo_sin_old_rev_tl_op'),
 (0.016569766119810248, 'tot_hi_cred_lim'),
 (0.016245337042536973, 'mths_since_recent_inq'),
 (0.01567288792341562, 'total_bal_ex_mort'),
 (0.015126511120291574, 'out_prncp_inv'),
 (0.01480837522826443, 'avg_cur_bal'),
 (0.014715058347461712, 'dti'),
 (0.01457874746980907, 'all_util'),
 (0.014572545074118079, 'tot_cur_bal'),
 (0.013966051770289771, 'total_bal_il'),
 (0.01396355899317676, 'mths_since_rcnt_il'),
 (0.013844089334101702, 'tota

### Easy Ensemble Classifier

In [211]:
# Train the Classifier
#model = EnsembleClassifier(solver = 'lbfgs', random_state = 1)
#model.fit(X_resampled, y_resampled)

from imblearn.ensemble import EasyEnsembleClassifier
ee = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee.fit(X_train_scaled, y_train)



EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [212]:
# Calculated the balanced accuracy score
#from sklearn.metrics import balanced_accuracy_score
#balanced_accuracy_score(y_test, y_pred)

y_pred = ee.predict(X_test_scaled)
ee_score = balanced_accuracy_score(y_test, y_pred)
print(ee_score)

#y_pred = ee.predict(X_test_scaled)
#ee_acc_score = balanced_accuracy_score(y_test, y_pred)
#print(f"Balanced Accuracy Score (Easy Ensemble Classifier): {ee_acc_score}")

0.7579972281647469


In [213]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

ee_cm = confusion_matrix(y_test, y_pred)
#y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   63,    24],
       [ 3563, 13555]], dtype=int64)

In [214]:
# Print the imbalanced classification repor
from imblearn.metrics import classification_report_imbalanced
ee_report=classification_report_imbalanced(y_test, y_pred)
print(ee_report)



                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.72      0.79      0.03      0.76      0.57        87
   low_risk       1.00      0.79      0.72      0.88      0.76      0.58     17118

avg / total       0.99      0.79      0.72      0.88      0.76      0.58     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier.

2. Which model had the best recall score?

    Balanced random forest

3. Which model had the best geometric mean score?

  Easy ensemble

4. What are the top three features?

  total_rec_prncp
  last_pymnt_amnt
  total_pymnt
  